In [1]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from speedy_utils.all import *

setup_logger("D")

DEBUG    | utils_print.py:265 (setup_logger) - Logging set to DEBUG


In [33]:





def slow_func(x):
    # try:
    time.sleep(1)  # sleeps for `x` seconds
    logger.info(f"{x=}")
    return x
    # except KeyboardInterrupt:
    #     logger.error("KB Interrupted")
    # except Exception as e:
    #     logger.error(f"Interrupted: {e}")
        

# write a decorator that raises an exception if the function is interrupted by the user


slow_func(1)

INFO     | 3873382941.py:4 (slow_func) - x=1


1

In [ ]:
setup_logger("D")

results = None
results = multi_thread(
    func=slow_func,
    orig_inputs=range(5),
    workers=2,
    desc="Demo",
    stop_on_error=True,
    # max_task_seconds=0.1
)

DEBUG    | utils_print.py:265 (setup_logger) - Logging set to DEBUG


Demo:   0%|                                                       | 0/5 [00:00<?, ?it/s]

INFO     | 3873382941.py:4 (slow_func) - x=1
INFO     | 3873382941.py:4 (slow_func) - x=0
ERROR    | parallel.py:29 (g) - Error at index 1: <class 'TimeoutError'>: Task exceeded 0.1 seconds


Demo:  20%|█████████▍                                     | 1/5 [00:01<00:04,  1.02s/it]

ERROR    | thread.py:170 (_execute_tasks_in_parallel) - Stop event set. Stopping execution.
DEBUG    | thread.py:172 (_execute_tasks_in_parallel) - Cancelling future <Future at 0x175e09400 state=running>
DEBUG    | thread.py:172 (_execute_tasks_in_parallel) - Cancelling future <Future at 0x175de5280 state=finished returned tuple>
DEBUG    | thread.py:172 (_execute_tasks_in_parallel) - Cancelling future <Future at 0x175e09130 state=running>
DEBUG    | thread.py:172 (_execute_tasks_in_parallel) - Cancelling future <Future at 0x175e082f0 state=cancelled>


Demo:  20%|█████████▍                                     | 1/5 [00:01<00:04,  1.03s/it]


WARNING  | thread.py:180 (_execute_tasks_in_parallel) - An error occurred during task execution. Next step: cleanup and return.
ERROR    | thread.py:305 (multi_thread) - [MAIN PROCESS] Keyboard interrupt detected. Stopping execution. Terminating child processes.


TimeoutError: Task exceeded 0.1 seconds

Traceback (most recent call last):
  File "/Users/anhvth/gitprojects/speedy_utils/speedy_utils/multi_worker/thread.py", line 67, in _process_single_task
    ret = _run_with_timeout(func, max_task_seconds, **inp)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/anhvth/gitprojects/speedy_utils/speedy_utils/multi_worker/thread.py", line 48, in _run_with_timeout
    return future.result(timeout=timeout)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/anhvth/miniconda3/lib/python3.12/concurrent/futures/_base.py", line 458, in result
    raise TimeoutError()
TimeoutError


In [61]:
results